In [1]:
GLOBAL_SEED = 7532

from numpy.random import seed
seed(GLOBAL_SEED)
from tensorflow import set_random_seed
set_random_seed(GLOBAL_SEED)

import numpy as np
import pandas as pd

from keras.models import Model, Sequential, load_model
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, MaxPool2D
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


Read in the metadata.

In [2]:
train_set = pd.read_csv('train_set_metadata.csv')
valid_set = pd.read_csv('valid_set_metadata.csv')
test_set = pd.read_csv('test_set_metadata.csv')

Set the main constants.

In [3]:
INPUT_SHAPE = (256, 384, 3)

LEARNING_RATE = 0.0001
N_EPOCHS = 100
BATCH_SIZE = 32

Read in the VGG16 model without the top part.

In [4]:
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 384, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 384, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 384, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 192, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 192, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 192, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 96, 128)       0         
__________

Freeze the convolutional layers.

In [5]:
for layer in vgg16_model.layers:
    layer.trainable = False

Create the model top.

In [6]:
def create_model_top(input_shape):
    model = Sequential()
    
    dropout_rate = 0.25
    
    model.add(Flatten())
    model.add(Dropout(rate=dropout_rate, seed=GLOBAL_SEED))
    model.add(Dense(units=1024, activation='relu'))
    model.add(Dropout(rate=dropout_rate, seed=GLOBAL_SEED))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dropout(rate=dropout_rate, seed=GLOBAL_SEED))
    model.add(Dense(5, activation='softmax'))
    
    return model

Attach the model top to the VGG16 model.

In [7]:
model_top = create_model_top(vgg16_model.outputs[0].get_shape().as_list()[1:])
model = Model(inputs=vgg16_model.inputs, outputs=model_top(vgg16_model.outputs[0]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 384, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 384, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 384, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 192, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 192, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 192, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 96, 128)       0         
__________

Verify that the appropriate layers are frozen.

In [8]:
for layer in model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True


Create the f1 metric to be used during model training.

In [9]:
class F1_Metric(Callback):
    def on_train_begin(self, logs={}):
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs={}):
        predict = np.round(np.asarray(self.model.predict(self.validation_data[0])))
        targ = self.validation_data[1]

        self.f1_scores.append(f1_score(targ, predict, average='weighted'))
        
        print(f' val_f1: {self.f1_scores[-1]}')
        
        return
    
f1_metric = F1_Metric()

Compute the class weights to be used during model training in order to fight the class imbalances.

In [10]:
class_values = train_set['category'].values - 1

classes = np.unique(class_values)
weights = compute_class_weight('balanced', np.unique(class_values), class_values)

class_weights = dict(zip(classes, weights))
class_weights

{0: 0.589811320754717,
 1: 1.0717714285714286,
 2: 1.3650655021834062,
 3: 1.5028846153846154,
 4: 1.0271631982475355}

Train the model utilizing Adam optimizer, learning rate reduction on plateau, class weights and data augmentation.

In [11]:
# Prepare the training and validation data
X_train = np.load('train_set_hmgd_arr_256_384_VGG16.npy')
y_train = to_categorical(train_set['category'].values - 1)

X_valid = np.load('valid_set_hmgd_arr_256_384_VGG16.npy')
y_valid = to_categorical(valid_set['category'].values - 1)


# Create and compile the model
adam = Adam(lr=LEARNING_RATE)
model.compile(optimizer=adam, loss='categorical_crossentropy')


# Initialize callbacks
checkpoint = ModelCheckpoint('model_006_epoch_{epoch:03d}.hdf5', monitor='val_loss', 
                             save_best_only=False, save_weights_only=False)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6)
#early_stopping = EarlyStopping(monitor='val_loss', patience=12)    
callback_list = [f1_metric, checkpoint, lr_reduction]#, early_stopping]


# data augmentation
data_gen = ImageDataGenerator(rotation_range=20, 
                              width_shift_range=0.2, 
                              height_shift_range=0.2, 
                              horizontal_flip=True)

steps_per_epoch = int(len(X_train)) / BATCH_SIZE

model.fit_generator(data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=N_EPOCHS,
                    callbacks=callback_list,
                    validation_data=(X_valid, y_valid),
                    class_weight=class_weights,
                    workers=4, 
                    verbose=2)

Epoch 1/100
 - 157s - loss: 6.0217 - val_loss: 1.9538
 val_f1: 0.7867220992517062
Epoch 2/100
 - 123s - loss: 3.1127 - val_loss: 1.4969
 val_f1: 0.8093275017072427
Epoch 3/100
 - 123s - loss: 2.2001 - val_loss: 1.1362
 val_f1: 0.8038195868418624
Epoch 4/100
 - 123s - loss: 1.5973 - val_loss: 0.6016
 val_f1: 0.8333116403876368
Epoch 5/100
 - 123s - loss: 1.1584 - val_loss: 0.4450
 val_f1: 0.8243335125863693
Epoch 6/100
 - 123s - loss: 0.8943 - val_loss: 0.4133
 val_f1: 0.8401069292156482
Epoch 7/100
 - 124s - loss: 0.8075 - val_loss: 0.4064
 val_f1: 0.8418190571308317
Epoch 8/100
 - 125s - loss: 0.6824 - val_loss: 0.3863
 val_f1: 0.8527781338419472
Epoch 9/100
 - 124s - loss: 0.6636 - val_loss: 0.4361
 val_f1: 0.8242042283800883
Epoch 10/100
 - 123s - loss: 0.6153 - val_loss: 0.3384
 val_f1: 0.8774729098883726
Epoch 11/100
 - 124s - loss: 0.6054 - val_loss: 0.3517
 val_f1: 0.8688256518439901
Epoch 12/100
 - 123s - loss: 0.5760 - val_loss: 0.3414
 val_f1: 0.8663448186890806
Epoch 13/100


Epoch 100/100
 - 123s - loss: 0.2005 - val_loss: 0.2680
 val_f1: 0.9092119515222542


In [12]:
# free up memory
del X_train
del X_valid

Read in the test data as well as the best model with respect to the f1 score.

In [14]:
X_test = np.load('test_set_hmgd_arr_256_384_VGG16.npy')
model = load_model('model_006_epoch_088.hdf5')

Compute the predictions and prepare the submission.

In [16]:
test_pred_classes = model.predict(X_test).argmax(axis=-1) + 1
test_pred_classes

array([4, 4, 4, ..., 4, 4, 2], dtype=int64)

In [17]:
sample_submission = pd.read_csv('data/sample_submission.csv')
submission = sample_submission.copy()

In [18]:
submission['category'] = test_pred_classes
submission['category'].value_counts()

1    905
5    541
2    492
3    391
4    351
Name: category, dtype: int64

In [19]:
submission.head()

,image,category
0,1007700.jpg,4
1,1011369.jpg,4
2,1051155.jpg,4
3,1062001.jpg,2
4,1069397.jpg,4


In [20]:
submission.to_csv('model_006_submission_01.csv', index=False)

In [ ]:
#                      Epoch 88
# Validation score:  0.9094737873
# Leaderboard score: 0.9155052841